In [ ]:
import csv
import sqlite3


In [ ]:
conn = sqlite3.connect('casestudy.db')


In [ ]:
cursor = conn.cursor()


In [ ]:
cursor.execute('''CREATE TABLE IF NOT EXISTS actual (
                    Material_Description VARCHAR(255),
                    Plant VARCHAR(255),
                    Period INTEGER,
                    Year INTEGER,
                    Amount_in_LC REAL,
                    Quantity INTEGER
                )''')

In [ ]:
cursor.execute('''CREATE TABLE IF NOT EXISTS target (
                    Year INTEGER,
                    Period INTEGER,
                    Plant VARCHAR(255),
                    Material_Number VARCHAR(255),
                    Target_Value_in_LC REAL,
                    Target_Quantity INTEGER
                )''')

In [ ]:
cursor.execute('''CREATE TABLE IF NOT EXISTS price (
                    Plant VARCHAR(255),
                    Material_Description VARCHAR(255),
                    Price_per_case REAL
                )''')

In [ ]:
cursor.execute('''CREATE TABLE IF NOT EXISTS bncr (
                    Material_Number VARCHAR(255),
                    Bottle VARCHAR(255),
                    Crate VARCHAR(255)
                )''')

In [ ]:
with open('asset/actuals.csv', 'r') as file:
    csv_data = csv.reader(file)
    next(csv_data)  # Skip the header row if it exists
    cursor.executemany('''INSERT INTO actual VALUES (?, ?, ?, ?, ?, ?)''', csv_data)

In [ ]:
with open('asset/targets.csv', 'r') as file:
    csv_data = csv.reader(file)
    next(csv_data)  # Skip the header row if it exists
    cursor.executemany('''INSERT INTO target VALUES (?, ?, ?, ?, ?, ?)''', csv_data)

In [ ]:
with open('asset/price.csv', 'r') as file:
    csv_data = csv.reader(file)
    next(csv_data)  # Skip the header row if it exists
    cursor.executemany('''INSERT INTO price VALUES (?, ?, ?)''', csv_data)

In [ ]:
with open('asset/bncr.csv', 'r') as file:
    csv_data = csv.reader(file)
    next(csv_data)  # Skip the header row if it exists
    cursor.executemany('''INSERT INTO bncr VALUES (?, ?, ?)''', csv_data)

In [ ]:
cursor.execute('SELECT COUNT(*) FROM actual')

# Fetch the result
row_count = cursor.fetchone()[0]

print("Number of rows before:", row_count)

cursor.execute('''
    DELETE FROM actual
    WHERE rowid NOT IN (
        SELECT MIN(rowid)
        FROM actual
        GROUP BY Material_Description, Plant, Period, Year, Amount_in_LC, Quantity
    )
''')

cursor.execute('''
    DELETE FROM actual
    WHERE Amount_in_LC < 0
''')

cursor.execute('''
    DELETE FROM actual
    WHERE Quantity < 0
''')

cursor.execute('SELECT COUNT(*) FROM actual')

# Fetch the result
row_count = cursor.fetchone()[0]

print("Number of rows after:", row_count)


In [ ]:
cursor.execute('SELECT COUNT(*) FROM target')

# Fetch the result
row_count = cursor.fetchone()[0]

print("Number of rows before:", row_count)

cursor.execute('''
    DELETE FROM target
    WHERE rowid NOT IN (
        SELECT MIN(rowid)
        FROM target
        GROUP BY Year, Period, Plant, Material_Number, Target_Value_in_LC, Target_Quantity
    )
''')

cursor.execute('SELECT COUNT(*) FROM target')

# Fetch the result
row_count = cursor.fetchone()[0]

print("Number of rows after:", row_count)

In [ ]:
cursor.execute('SELECT COUNT(*) FROM price')

# Fetch the result
row_count = cursor.fetchone()[0]

print("Number of rows before:", row_count)

cursor.execute('''
    DELETE FROM price
    WHERE rowid NOT IN (
        SELECT MIN(rowid)
        FROM price
        GROUP BY Plant, Material_Description, Price_per_case
    )
''')

cursor.execute('SELECT COUNT(*) FROM price')

# Fetch the result
row_count = cursor.fetchone()[0]

print("Number of rows after:", row_count)

In [ ]:
cursor.execute('SELECT COUNT(*) FROM bncr')

# Fetch the result
row_count = cursor.fetchone()[0]

print("Number of rows before:", row_count)

cursor.execute('''
    DELETE FROM bncr
    WHERE rowid NOT IN (
        SELECT MIN(rowid)
        FROM bncr
        GROUP BY Material_Number, Bottle, Crate
    )
''')

cursor.execute('SELECT COUNT(*) FROM bncr')

# Fetch the result
row_count = cursor.fetchone()[0]

print("Number of rows after:", row_count)

In [ ]:
cursor.execute("SELECT * FROM actual LIMIT 10")
columns = cursor.fetchall()

for column in columns:
    print(column)

In [ ]:
cursor.execute("SELECT * FROM target LIMIT 10")
columns = cursor.fetchall()

for column in columns:
    print(column)

In [ ]:
cursor.execute("SELECT * FROM price LIMIT 10")
columns = cursor.fetchall()

for column in columns:
    print(column)

In [ ]:
cursor.execute("SELECT * FROM bncr LIMIT 10")
columns = cursor.fetchall()

for column in columns:
    print(column)

In [ ]:
cursor.execute('''
    SELECT actual.plant, actual.Amount_in_LC, actual.Quantity, target.Target_Value_in_LC, target.Target_Quantity
    FROM actual
    LEFT JOIN target ON actual.plant = target.plant
''')

result = cursor.fetchall()
for row in result:
    print(row)

In [ ]:
cursor.execute('''
    SELECT actual.Plant, actual.Material_Description, actual.Amount_in_LC, target.Target_Value_in_LC,
        actual.Quantity, target.Target_Quantity,
        (actual.Amount_in_LC - target.Target_Value_in_LC) AS Amount_Variance,
        (actual.Quantity - target.Target_Quantity) AS Quantity_Variance
    FROM actual
    LEFT JOIN target ON actual.Plant = target.Plant
''')

# Fetch all the rows from the result set
variance_data = cursor.fetchall()

# Create a new table to store the variance analysis results
cursor.execute('''
    CREATE TABLE IF NOT EXISTS variance_table (
        Plant VARCHAR(255),
        Material_Description VARCHAR(255),
        Amount_in_LC REAL,
        Target_Value_in_LC REAL,
        Quantity INTEGER,
        Target_Quantity INTEGER,
        Amount_Variance REAL,
        Quantity_Variance INTEGER
    )
''')

# Insert the variance analysis data into the new table
cursor.executemany('''
    INSERT INTO variance_table (Plant, Material_Description, Amount_in_LC, Target_Value_in_LC,
        Quantity, Target_Quantity, Amount_Variance, Quantity_Variance)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
''', variance_data)

# Delete duplicate rows
cursor.execute('''
    DELETE FROM variance_table
    WHERE rowid NOT IN (
        SELECT MIN(rowid)
        FROM variance_table
        GROUP BY Plant, Material_Description, Amount_in_LC, Target_Value_in_LC, Quantity, Target_Quantity, Amount_Variance, Quantity_Variance
    )
''')


In [ ]:
# Execute the SELECT query
cursor.execute('SELECT COUNT(*) FROM variance_table')

# Fetch all the rows from the result set
result = cursor.fetchone()

# Extract the count value from the result
num_rows = result[0]

# Print the number of rows
print("Number of rows:", num_rows)


In [ ]:
# Get the schema of the variance_table
cursor.execute("PRAGMA table_info(variance_table)")

# Fetch all the rows from the result set
schema_data = cursor.fetchall()

# Print the schema of the variance_table
for row in schema_data:
    print(row)


In [ ]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS variance_data_analysis (
        Material_Number TEXT,
        Plant TEXT,
        Category TEXT,
        Period TEXT,
        Year INTEGER,
        Actual_Amount_in_LC REAL,
        Actual_Quantity INTEGER,
        Target_Value_in_LC REAL,
        Target_Quantity INTEGER,
        Variance_Amount REAL,
        Variance_Quantity INTEGER
    )
''')


In [ ]:
cursor.execute('''
    INSERT INTO variance_data_analysis (
        Material_Number,
        Plant,
        Category,
        Period,
        Year,
        Actual_Amount_in_LC,
        Actual_Quantity,
        Target_Value_in_LC,
        Target_Quantity,
        Variance_Amount,
        Variance_Quantity
    )
    SELECT
        target.Material_Number,
        actual.Plant,
        CASE
            WHEN actual.Material_Description = 'Bottle' THEN 'Bottle'
            WHEN actual.Material_Description = 'Keg' THEN 'Keg'
            WHEN actual.Material_Description = 'Crate' THEN 'Crate'
        END AS Category,
        actual.Period,
        actual.Year,
        actual.Amount_in_LC,
        actual.Quantity,
        target.Target_Value_in_LC,
        target.Target_Quantity,
        actual.Amount_in_LC - target.Target_Value_in_LC AS Variance_Amount,
        actual.Quantity - target.Target_Quantity AS Variance_Quantity
    FROM
        actual
    LEFT JOIN
        target ON actual.Plant = target.Plant
''')

conn.commit()

# Print the number of inserted rows
cursor.execute('''
    DELETE FROM variance_data_analysis
    WHERE rowid = (
        SELECT rowid
        FROM variance_data_analysis
        LIMIT 1
    )
''')
# conn.commit()
print(f"Number of inserted rows: {cursor.rowcount}")
# Fetch all the inserted rows
cursor.execute('SELECT * FROM variance_data_analysis')
result = cursor.fetchall()

# Print the inserted rows
for row in result:
    print(row)


In [ ]:
cursor.execute("SELECT * FROM variance_table")

# Fetch all the rows from the result set
rows = cursor.fetchall()

# Define the output CSV file path
csv_file_path = "results/variance_table.csv"

# Open the CSV file in write mode
with open(csv_file_path, "w", newline="") as csvfile:
    # Create a CSV writer object
    csv_writer = csv.writer(csvfile)

    # Write the header row
    header = [column[0] for column in cursor.description]
    csv_writer.writerow(header)

    # Write the data rows
    csv_writer.writerows(rows)

print(f"Data exported to {csv_file_path} successfully.")

In [ ]:
cursor.execute("SELECT * FROM variance_data_analysis")
# Fetch all the rows from the result set
rows = cursor.fetchall()
# Define the output CSV file path
csv_file_path = "results/variance_data_analysis.csv"
# Open the CSV file in write mode
with open(csv_file_path, "w", newline="") as csvfile:
    # Create a CSV writer object
    csv_writer = csv.writer(csvfile)

    # Write the header row
    header = [column[0] for column in cursor.description]
    csv_writer.writerow(header)

    # Write the data rows
    csv_writer.writerows(rows)

print(f"Data exported to {csv_file_path} successfully.")

In [ ]:
categories = ['Bottle', 'Keg', 'Crate']
plants = ['AA', 'AB', 'AC','AD', 'AE','AF', 'AG']

with open('results/trend_analysis.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['Category', 'Plant', 'Year', 'Period', 'Total Quantity'])

    for category in categories:
        for plant in plants:
            query = f"SELECT Year, Period, SUM(Quantity) FROM actual WHERE Material_Description LIKE '{category}%' AND Plant = '{plant}' GROUP BY Year, Period"
            cursor.execute(query)
            results = cursor.fetchall()

            for result in results:
                year, period, total_quantity = result
                writer.writerow([category, plant, year, period, total_quantity])


In [ ]:
conn.execute('''
CREATE VIEW consolidated_actuals10 AS
SELECT
  actual.Material_Description,
  actual.Plant,
  CASE
    WHEN actual.Material_Description LIKE '%Bottle%' THEN price.Price_per_case
    ELSE 0
  END AS `Bottle Price`,
  CASE
    WHEN actual.Material_Description LIKE '%Crate%' THEN price.Price_per_case
    ELSE 0
  END AS `Crate Price`,
  CASE
    WHEN actual.Material_Description LIKE '%Bottle%' THEN actual.Amount_in_LC * price.Price_per_case
    WHEN actual.Material_Description LIKE '%Crate%' THEN actual.Amount_in_LC * price.Price_per_case
    ELSE 0
  END AS `Bottle Rands`,
  CASE
    WHEN actual.Material_Description LIKE '%Bottle%' THEN 0
    WHEN actual.Material_Description LIKE '%Crate%' THEN actual.Amount_in_LC * price.Price_per_case
    ELSE 0
  END AS `Crate Rands`
FROM actual
JOIN price ON actual.Material_Description = price.Material_Description AND actual.Plant = price.Plant
JOIN bncr ON actual.Material_Description = bncr.Material_Number
''')


for row in conn.execute("SELECT * FROM consolidated_actuals10 WHERE Material_Description LIKE '%crate%' OR Material_Description LIKE '%keg%' OR Material_Description LIKE '%bottle%'"):
    print(row)